In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

API Key loaded: True


In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [4]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [5]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [6]:

raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# pick sensible values
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,      # > 0
    chunk_overlap=100     # >= 0 and < chunk_size
)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1088, which is longer than the specified 1000
Created a chunk of size 1189, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1057, which is longer than the specified 1000
Created a chunk of size 1270, which is longer than the specified 1000
Created a chunk of size 1635, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

In [7]:


documents[0]



Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [8]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [9]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='08c6b31e-bd70-4bb3-a98b-c102ee410b2c', metadata={'source': 'tagged_description.txt'}, page_content="9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.\n9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."),
 Document(id='90805362-2dbe-49e4-85f0-9753e73b0c8a', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine ar

In [10]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [11]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [12]:
retrieve_semantic_recommendations("A book to teach children about nature")

isbn13      isbn10  \
30    9780006646006  000664600X   
260   9780060852559  0060852550   
324   9780060959036  0060959037   
399   9780062516374  006251637X   
413   9780064405850  0064405850   
424   9780064410724  0064410722   
442   9780067575208  006757520X   
879   9780152045739  0152045732   
1077  9780240806082  0240806085   
1220  9780312315733  0312315732   
1639  9780374422080  0374422087   
1641  9780374517885  0374517886   
1696  9780375713934  037571393X   
1738  9780375806704  0375806709   
1808  9780385319270  0385319274   
1905  9780393311037  0393311031   
2102  9780439227148  0439227143   
2139  9780439736466  0439736463   
2261  9780446518628  044651862X   
2301  9780446677493  0446677493   
2409  9780451456816  0451456815   
2497  9780486256429  0486256421   
2503  9780486276809  0486276805   
2858  9780590032490  0590032496   
2870  9780590956161  0590956167   
2921  9780618446872  0618446877   
3197  9780689707490  0689707495   
3201  9780689823824  0689823827   
3213  9780689860072  0689860072   
3470  9780747566755  0747566755   
3576  9780762408283  0762408286   
3578  9780762419227  0762419229   
3582  9780763621643  0763621641   
3591  9780764122781  0764122789   
3604  9780764584657  0764584650   
3676  9780767908184  076790818X   
3747  9780786808069  0786808063   
3749  9780786808380  0786808381   
3751  9780786808717  0786808713   
3764  9780786818723  0786818727   
3767  9780786849734  0786849738   
3920  9780810117495  0810117495   
4185  9780878937202  087893720X   
4231  9780898599596  0898599598   
4264  9780941807555  094180755X   
4331  9781400033850  1400033853   
4359  9781400051533  1400051533   
4455  9781405201711  1405201711   
4780  9781584850557  1584850558   
4809  9781586857721  158685772X   

                                                  title  \
30                                   Ocean Star Express   
260                          Animal, Vegetable, Miracle   
324                                     Prodigal Summer   
399                            Rainforest Home Remedies   
413            Strawberry Girl 60th Anniversary Edition   
424                                  Four Stupid Cupids   
442                                 The Sense of Wonder   
879   The Little Red Lighthouse and the Great Gray B...   
1077                          Directing the Documentary   
1220                                    Little Children   
1639                             Everything on a Waffle   
1641                                           Sammlung   
1696                                 John James Audubon   
1738                              The Phantom Tollbooth   
1808                  There's No Such Place as Far Away   
1905                       Hen's Teeth and Horse's Toes   
2102                               The Call of the Wild   
2139                          Where We Are, what We See   
2261                             The Celestine Prophecy   
2301                    Hope is the Thing with Feathers   
2409                                  Not of Woman Born   
2497                                          Galapagos   
2503                    The Secret Garden Coloring Book   
2858                                        The witches   
2870                           The Meanest Thing to Say   
2921                                   Dark Star Safari   
3197                  Cloudy With a Chance of Meatballs   
3201                         A Child's Garden of Verses   
3213                                       Dragonsinger   
3470                             The Story of Mr Sommer   
3576                         The Mini Zen Gardening Kit   
3578                           God Created The Integers   
3582                                       Castle Diary   
3591                                    Aquatic Turtles   
3604                              Astronomy For Dummies   
3676               A Short History of Nearly Everything   
3747                Baby Einstein: Neighborh